In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import cPickle as pickle

In [2]:
def read_file_and_score(num_comps):
    w_citywide = pd.read_csv('../../Data/Craigslist/comp_variants/beds_%s_w_city.csv' % num_comps)
    wo_citywide = pd.read_csv('../../Data/Craigslist/comp_variants/beds_%s_wo_city.csv' % num_comps)

    print '-'*30
    print 'Comps: %s' % num_comps
    print 'MAE w city, median: ', mean_absolute_error(w_citywide.price, w_citywide.comp_median_price)
    print 'MSE w city, median: ', mean_squared_error(w_citywide.price, w_citywide.comp_median_price)

    print 'MAE w city, mean: ', mean_absolute_error(w_citywide.price, w_citywide.comp_mean_price)
    print 'MSE w city, mean: ', mean_squared_error(w_citywide.price, w_citywide.comp_mean_price)

    print 'MAE wo city, median: ', mean_absolute_error(wo_citywide.price, wo_citywide.comp_median_price)
    print 'MSE wo city, median: ', mean_squared_error(wo_citywide.price, wo_citywide.comp_median_price)

    print 'MAE wo city, median: ', mean_absolute_error(wo_citywide.price, wo_citywide.comp_mean_price)
    print 'MSE wo city, median: ', mean_squared_error(wo_citywide.price, wo_citywide.comp_mean_price)
    print '\n'
    return None

In [3]:
comp_amts = [10, 20, 50, 100]
for n in comp_amts:
    read_file_and_score(n)

------------------------------
Comps: 10
MAE w city, median:  511.48449041
MSE w city, median:  687900.163042
MAE w city, mean:  527.818011516
MSE w city, mean:  653371.668307
MAE wo city, median:  510.957799902
MSE wo city, median:  686537.883125
MAE wo city, median:  527.272455871
MSE wo city, median:  651809.034555


------------------------------
Comps: 20
MAE w city, median:  531.435770481
MSE w city, median:  698706.030108
MAE w city, mean:  545.971454905
MSE w city, mean:  677316.370902
MAE wo city, median:  530.909079974
MSE wo city, median:  697343.750191
MAE wo city, median:  545.425899259
MSE wo city, median:  675753.73715


------------------------------
Comps: 50
MAE w city, median:  554.35369067
MSE w city, median:  725240.580354
MAE w city, mean:  566.170385945
MSE w city, mean:  707508.1448
MAE wo city, median:  553.827000163
MSE wo city, median:  723878.300437
MAE wo city, median:  565.624830299
MSE wo city, median:  705945.511048


------------------------------
Comps

In [4]:
# now that we have cross-validated number of comps
df = pd.read_csv('../../Data/Craigslist/featurized_final.csv')

In [5]:
X = df.drop(['Unnamed: 0','Unnamed: 0.1','id', 'heading', 'body', 'region','neighborhood', 'accuracy','date','flagged','comps_found', 'fewer_than_five'], axis=1)
X['apt'].fillna(0, inplace=True)
X['house'].fillna(0, inplace=True)
X = X[X['sqft'].notnull()]

In [6]:
y = df.price

In [7]:
X.drop(['lat', 'long', 'price'], axis=1, inplace=True)

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20895 entries, 7 to 61514
Data columns (total 8 columns):
beds                 20895 non-null float64
baths                20895 non-null float64
parking              20895 non-null int64
sqft                 20895 non-null float64
apt                  20895 non-null float64
house                20895 non-null float64
comp_median_price    20895 non-null float64
comp_mean_price      20895 non-null float64
dtypes: float64(7), int64(1)
memory usage: 1.4 MB


In [9]:
X.beds.value_counts()

1    6879
0    5970
2    5757
3    1840
4     366
5      65
6      13
8       5
dtype: int64

In [10]:
X[X.beds==1]['baths'].value_counts()


1    6675
2     201
3       3
dtype: int64

In [11]:
print np.median(X[X.beds==1]['sqft'])
print np.mean(X[X.beds==1]['sqft'])

725.0
760.513010612


In [12]:
X[X.beds==1]['apt'].value_counts()

1    5062
0    1817
dtype: int64

In [13]:
X[X.beds==1]['parking'].value_counts()

0    3010
4    2299
1     986
3     408
2     176
dtype: int64

In [14]:
sample_for_grid = np.array([1, 1, 0, 725.0, 1, 0])

In [15]:
df2 = pd.read_csv('../../Data/Craigslist/featurized_full_3.csv')

In [16]:
df2.head()

,Unnamed: 0,Unnamed: 0.1,id,heading,body,price,lat,long,region,neighborhood,...,parking,sqft,apt,accuracy,date,flagged,house,comp_median_price,comps_found,fewer_than_five
0,0,0,1418103620,SRO Type Room In Chinatown,\n **Open House on Friday 10/3 3pm to 5pm **\n...,675,37.795400,-122.406086,sfc,financial district,...,1,NaN,1,8,2014-09-29 04:40:43,NaN,NaN,800.0,10,0
1,1,1,1418876214,Top Floor Fantastic Totally Remodeled Marina S...,\n Contact info: Walt\n Top Floor Totally Remo...,2350,37.801036,-122.434535,sfc,marina / cow hollow,...,1,NaN,1,8,2014-09-29 14:24:16,NaN,NaN,2195.0,10,0
2,2,2,1418876409,Just Renovated. Small Studio. Across St from T...,\n Open House Sunday 3:00-3:20\nJust Renovated...,1625,37.776789,-122.415409,sfc,SOMA / south beach,...,1,NaN,1,8,2014-09-29 14:22:43,NaN,NaN,1610.0,10,0
3,3,3,1419161628,Large Junior 1 BR / Close to Downtown / Hardwo...,\n Apartment Features:\n- Jr. 1 bedroom / 1 ba...,2495,37.784960,-122.416176,sfc,downtown / civic / van ness,...,1,NaN,1,8,2014-09-29 15:39:15,NaN,NaN,2022.5,10,0
4,4,4,1419200940,"Noe Valley gorgeous fully furnished, free util...",\n email to setup a showing\n* Free utilities\...,2895,37.752821,-122.424073,sfc,noe valley,...,0,NaN,1,8,2014-09-29 15:49:59,NaN,NaN,2750.0,10,0


In [17]:
X.shape

(20895, 8)

In [18]:
with open('../../Models/Craigslist/rf_craigslist.pkl') as f:
    rf = pickle.load(f)

KeyboardInterrupt: (None, <function _reconstructor at 0x10077ca28>, (<class 'sklearn.tree.tree.DecisionTreeRegressor'>, <type 'object'>, None))

In [ ]:
rf.fit(X.values, y.values)

In [ ]:
with open('../../Models/Craigslist/rf_craigslist.pkl', 'w') as f:
    pickle.dump(rf, f)

In [ ]:
grid = pd.read_csv('../../Data/sf_gps_grid.csv', header=None)

In [ ]:
grid.shape

In [ ]:
with open('../../Models/Craigslist/kdtree_full2.pkl') as f:
    kdtree = pickle.load(f)

In [ ]:
predictions = kdtree.query_radius()

In [19]:
sample_for_grid = np.array([[1, 1, 0, 725.0, 1, 0]] * 40000)

In [20]:
sample_for_grid = pd.DataFrame(sample_for_grid, columns = ['beds', 'baths', 'parking', 'sqft', 'apt', 'house'] )

In [ ]:
grid_gps = pd.read_csv('../../Data/sf_gps_grid.csv', header=None)

In [ ]:
grid_gps.columns = ['lat', 'long']

In [ ]:
sample_for_grid = pd.merge(sample_for_grid, grid_gps, left_index=True, right_index=True)

In [ ]:
sample_for_grid.to_csv('../../Data/Craigslist/sf_grid_points.csv', encoding='utf-8')